# Data Exploration
## Goodreads Books Dataset

In [2]:
import pandas as pd
import numpy as np
import gzip
import json

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

sns.set_theme(style="darkgrid")

In [3]:
def load_data(file_name, head = 500):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            
            # break if reaches the 100th line
            if (head is not None) and (count > head):
                break
    return pd.DataFrame(data)

In [4]:
# Load samples
books = load_data('data/raw/goodreads_books.json.gz', head=10000)
reviews = load_data('data/raw/goodreads_reviews_dedup.json.gz', head=50000)

In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   isbn                  10001 non-null  object
 1   text_reviews_count    10001 non-null  object
 2   series                10001 non-null  object
 3   country_code          10001 non-null  object
 4   language_code         10001 non-null  object
 5   popular_shelves       10001 non-null  object
 6   asin                  10001 non-null  object
 7   is_ebook              10001 non-null  object
 8   average_rating        10001 non-null  object
 9   kindle_asin           10001 non-null  object
 10  similar_books         10001 non-null  object
 11  description           10001 non-null  object
 12  format                10001 non-null  object
 13  link                  10001 non-null  object
 14  authors               10001 non-null  object
 15  publisher             10001 non-null

-> Only object type columns so we need to format the dataset.\
-> Missing values are not counted because of object type.

In [6]:
books.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0312853122,1,[],US,,"[{'count': '3', 'name': 'to-read'}, {'count': ...",,false,4.00,,...,9,,1984,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3,5400751,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,6,[],US,,"[{'count': '2634', 'name': 'to-read'}, {'count...",,false,3.23,B000FC0PBC,...,10,Abridged,2001,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10,1323437,Good Harbor,Good Harbor
2,,7,[189911],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,false,4.03,,...,,Book Club Edition,1987,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140,8948723,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,...,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
4,0850308712,5,[],US,,"[{'count': '32', 'name': 'to-read'}, {'count':...",,false,3.40,,...,,,,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15,278577,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...


Let's add one columne "popularity rate" to sort our dataset.

In [7]:
popularity_rate = []
for index, row in books.iterrows():
    count = 0
    for cat in row['popular_shelves']:
        count += int(cat['count'])
    popularity_rate.append(count)

books['popularity_rate'] = popularity_rate

books.sort_values(by=['popularity_rate'], ascending=False, inplace=True)

books.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,popularity_rate
3373,9604533940,21,[],US,gre,"[{'count': '834959', 'name': 'to-read'}, {'cou...",,false,4.36,,...,,2008,https://www.goodreads.com/book/show/16281293,https://images.gr-assets.com/books/1354871653m...,16281293,142,878368,Η κλέφτρα των βιβλίων,Η κλέφτρα των βιβλίων,1006161
8096,8917161375,107,[],US,eng,"[{'count': '742446', 'name': 'to-read'}, {'cou...",,false,4.25,B01B173GA6,...,,2006,https://www.goodreads.com/book/show/1555826.Pr...,https://s.gr-assets.com/assets/nophoto/book/11...,1555826,710,3060926,Pride and Prejudice,Pride and Prejudice,997017
2693,3954180154,1,[],US,ger,"[{'count': '742446', 'name': 'to-read'}, {'cou...",,true,4.25,,...,1. Auflage,2012,https://www.goodreads.com/book/show/18000257-s...,https://images.gr-assets.com/books/1369944679m...,18000257,1,3060926,Stolz und Vorurteil,Stolz und Vorurteil,997017
9991,9188877663,5,[167817],US,swe,"[{'count': '525550', 'name': 'to-read'}, {'cou...",,false,4.45,,...,2,2001,https://www.goodreads.com/book/show/6015185-ha...,https://images.gr-assets.com/books/1465566233m...,6015185,96,4640799,"Harry Potter och de vises sten (Harry Potter, #1)","Harry Potter och de vises sten (Harry Potter, #1)",771435
3221,,12,[167817],US,eng,"[{'count': '525550', 'name': 'to-read'}, {'cou...",,true,4.45,,...,US Enhanced Edition,2015,https://www.goodreads.com/book/show/27421523-h...,https://images.gr-assets.com/books/1446575623m...,27421523,80,4640799,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Sorcerer's Stone (Harry P...,771435


In [12]:
#books.to_excel('data/excel/books10k.xlsx', sheet_name='head10k')
#reviews.to_excel('data/excel/reviews50k.xlsx', sheet_name='head50k')

### Data Cleaning And Format

In [ ]:
#Set missing values as NULL
books.replace('', np.NaN, inplace=True)

integers = ['text_reviews_count','ratings_count','publication_day','publication_month', 'publication_year','book_id','num_pages']
floats = ['average_rating']

#Change column types
for col in integers:
    books[col] = books[col].fillna(0)
    books[col] = books[col].astype(int)
for col in floats:
    books[col] = books[col].astype(float)

In [ ]:
sns.heatmap(books.isnull(), cbar=False)

### Item Dataset Exploration

In [ ]:
books.describe()

#### 1. Ratings and reviews

In [ ]:
ax = sns.boxplot(data=books[["ratings_count","text_reviews_count"]])
ax.set_ylim(0,200)

### 2. Categories

-> Language

In [ ]:
lang = books["language_code"].value_counts().sort_values(ascending=False)
#Display top 20 languages represented
lang.head(10)

#### a. Books categories

In [ ]:
all_categories = {}
for row in books['popular_shelves'].tolist():
    if row:
        for cat in row:
            if cat['name'] not in all_categories.keys():
                all_categories[cat['name']] = int(cat['count'])
            else:
                all_categories[cat['name']] += int(cat['count'])
print(len(all_categories))

About 120000 categories are represented.\
Some of them might be underepresented, overrepresented or just unrelevent.\
-> We need to choose which one we will consider as features.

In [ ]:
categories = pd.DataFrame({}, columns=['name', 'counts'])

categories['name'] = all_categories.keys()
categories['counts'] = all_categories.values()
categories.sort_values(by=['counts'], inplace=True, ascending=False)

In [ ]:
#Print the top 10 categories
categories.head(10)

One can see that some categories like "currently-reading", "owned", "books-i-own" aren't really describing the books.\
But let's keep it for the moment.


In [ ]:
#Print the bottom 10 categories
categories.tail(10)

One can see that there are lots of noises among the categories.\
Those ones are unrelevent, too specifics or just mistakes.\
-> We need to remove them from the dataset since it can't be a source of informations.

In [ ]:
categories['counts'].describe()

One can see that at least 75% of those categories are represented less than twice.\
It means that we can already eliminate those one.

In [ ]:
#Let's continue with the top 25%
categories = categories.head(int(round(len(categories)*25/1000, 0)))


print(len(categories))
categories.describe()

In [ ]:
#Print the bottom 10 categories
categories.tail(10)

In [ ]:
linked = []
for cat in categories['name']:
    if 'romance' in cat.lower():
        linked.append(cat)
print(len(linked))
print(linked)

One can see that many categories are too specific and contains sub-categories that a more represented. (exemple: Romance)\
We need to separate those categories.

Let's try to:\
-> remove duplicates (like "sci-fi" and "science-fiction")\
-> remove plurales (like "childrenS")\
-> remove categories that give informations we already have in the dataset. (like "series" or "ebook")\
-> remove non categorical labels (like "favorite", "books-iown" etc.)

In [ ]:
selected_cat = categories['name'].tolist()[:50]

print(selected_cat)

In [ ]:
to_remove = ['to-read', 'currently-reading', 'favorites', 'books-i-own', 'owned', 'favourites', 'series',
            'non-fiction', 'library', 'default','to-buy', 'childrens', 'ya', 'ebooks', 'dystopian', 'audiobooks',
            'ebook','my-books','nonfiction' ,'audiobook','i-own', 'my-library','owned-books','owned-books',
            'science-fiction', 'historical-fiction','novels','classics','young-adult','book-club']

for cat in to_remove:
    try:
        selected_cat.remove(cat)
    except ValueError:
        pass

selected_cat.append('novel')
selected_cat.append('young')

print(selected_cat)

Now not to loose information because of noise, we need to associate a lexical field to those categories.\
We will then look for similarities among the rest of the dataset to classify better the books. 

In [ ]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

stem_categories = {}

for cat in selected_cat:
    stem_categories[ps.stem(cat)] = cat

print(stem_categories)

#### b. Publisher

In [ ]:
publisher_count = books['publisher'].value_counts().sort_values(ascending=False)
print(len(publisher_count))
publisher_count.head(15)

There are too many different publishers to create features like we did for books categories.
Furthemore, publisher frequencies aren't that high.

-> Let's keep the column like that. We will then create a feature telling if it is the same publisher depending on the book we are comparing.

-> We'll do the same for "Author".

In [ ]:
author = books['authors'].apply(lambda x : 0 if not x else x[0]['author_id'])
author.head()

#### c. Date

We will use only the column "Year" to compare books between them.

In [ ]:
publication_year = books['publication_year'].replace(0, np.nan)

In [ ]:
missing_year = round(publication_year.isna().sum()/len(publication_year)*100, 2)
print( f"{missing_year}% of the values are missing.")

In [ ]:
publication_year.describe()

In [ ]:
publication_year.where(publication_year != np.NaN).sort_values().head(10)

Since that at least 75% of the books as been published after 2004, we can consider those values as outliers.

-> Let's set the minimum publication year as 1960.\
-> We will bring together all the values bellow this date to 1960.

Concerning the missing values, we could :
1. Assign them the average publication year value.
2. Set them as the minimum (1960).

-> At first, we'll choose the second option to avoid giving to this feature to much importance.

In [ ]:
publication_year.fillna(0, inplace=True)
publication_year[publication_year < 1960] = 1960
publication_year.head()

In [ ]:
# MinMax Normalization
publication_year = (publication_year - publication_year.min())/(publication_year.max()-publication_year.min())
publication_year.head()

#### d. Format

In [ ]:
format_count = books['format'].value_counts().sort_values(ascending=False)
print(len(format_count))
format_count.head(10)

In [ ]:
top_format = ['Paperback', 'Hardcover', 'ebook', 'Kindle Edition', 'Mass Market Paperback', 'Audiobook']
book_format = books['format']
book_format.fillna('Undefined', inplace=True)
book_format.replace('ebook','Ebook', inplace=True)
book_format = book_format.apply(lambda x : 'Audiobook' if 'Aud' in x else x)
book_format = book_format.apply(lambda x : 'Other' if x not in top_format and x != 'Undefined' else x)

book_format.head(5)

In [ ]:
book_format.value_counts().sort_values(ascending=False)

#### e. Series

In [ ]:
books['series'].head()

In [ ]:
is_series = books['series'].apply(lambda x: int(bool((x))))
is_series.head()

## 3. Sampling for Content Base Filtering

Now let's create a new dataset containing the most important columns to compare books.

It may include:\
-> Categories (most important one)\
-> Author\
-> Publisher\
-> Date\
-> Format\
-> Is it a serie of books ?\
-> ... ?

In [ ]:
features = pd.DataFrame({}, columns=['book_id']+selected_cat)

#Let's create a column for each categories
#Those columns will contain the percentage of counts
for index, row in books.iterrows():
    new_row = [0]*len(selected_cat)
    for cat in row['popular_shelves']:
        for stem in stem_categories.keys():
            if stem in cat['name']:
                new_row[selected_cat.index(stem_categories[stem])] = int(cat['count'])

    sum_count = 0
    for value in new_row:
        sum_count += value
    
    new_row = [x/sum_count if sum_count != 0 else x for x in new_row]
    
    new_row.insert(0, row['book_id'])

    features.loc[len(features)] = new_row

features['book_id'] = features['book_id'].astype(int)

features.head()

We need to check that the big majority of the books is well categories.

In [ ]:
nb_nocat = 0
for index, row in features.iterrows():
    for col in features.columns.tolist()[1:]:
        if row[col] != 0:
            break
    else:
        nb_nocat += 1

print(f"{round(nb_nocat/len(features)*100,2)}% of the books are not categorised.")

So we can see that a big majority of the books are well categorized.\
However, 15% is still a signifcant part and we might be able to improve that later.

For the moment, let's keep going with those categories.

In [ ]:
#Add the last features
features['author'] = author
features['publisher'] = books['publisher']
features['year'] = publication_year
features['format'] = book_format
features['is_series'] = is_series

features.head()

Now, we have a clean dataset we can start working with for a "Content Based Filtering" approach.

In [ ]:
#Export in the processed dataset in csv
features.to_csv('data/sample/item_item_filter.csv', index=False)